In [1]:
import numpy as np
import copy as cp
import subprocess as sp
import os as os
import shutil as sh
import MDAnalysis as mdana
import sys
from MDAnalysis.analysis.distances import distance_array
import pandas as pd
import mdtraj as md
import matplotlib
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix


/home/yadav129/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ref_structure="RAEF_Run1_50_npt_eq.pdb"
traj         ="RAEF_Run1_50_protein_final.xtc"
Name         ='RAEF_50_Run1'


#structure parameters
topology   = md.load(ref_structure).topology
trajectory = md.load(traj, top=ref_structure)
frames     = trajectory.n_frames #Number of frames
chains     = 50   #Number of chains
atoms      = int(topology.n_atoms/chains)#Number of atoms in each monomer 
AminoAcids = int(topology.n_residues/chains) #Number of residues per chain 

In [3]:
frames, AminoAcids

(20001, 15)

In [4]:
isum=1
atoms_list=[]
atomsperAminoAcid=[]
residue_list=[]

In [5]:
chains, frames, atoms*chains

(50, 20001, 1600)

In [6]:
uni = mdana.Universe(ref_structure,traj)
n,t = list(enumerate(uni.trajectory))[0]
box = t.dimensions[:6]

/home/yadav129/.local/lib/python3.8/site-packages/MDAnalysis/topology/PDBParser.py:317: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "


In [7]:
contact_sum = np.zeros((AminoAcids, AminoAcids))

max_distance=5.0
dist_arr    = []
contact_arr = []


label_arr= []
for tt in uni.trajectory[0::100]:
    atom_Groups = [[] for x in range(chains)]
    m_start=0
    for m in range(0,chains):
        m_end = atoms * (m+1)
        atom_Groups[m].extend([uni.select_atoms('bynum '+ str(m_start) + ':' + str(m_end) +' and name BB')])
        m_start = m_end + 1
    mySet = set([])    
    count = 0
    for i in range(chains-1):
        for j in range(i+1,chains):
            dist = np.round(distance_array(atom_Groups[i][0].positions,atom_Groups[j][0].positions,box),2)
            dist[dist <= max_distance] = 1
            dist[dist >= max_distance] = 0
            contact_sum = dist + contact_sum
            dist_arr.append(dist)
            contact_arr.append(contact_sum)
            label_arr.append(i)

In [8]:
def displayMatrix(matrix):     
    for row in matrix:
        for element in row:
            print(element, end =" ")
        print("\n")

In [9]:
#displayMatrix(dist_arr)

In [10]:
label_arr=np.asarray(label_arr)
dist_arr =np.asarray(dist_arr)
dist_arr.shape

(246225, 15, 15)

In [11]:
import h5py
with h5py.File(Name+'_data.h5', 'w') as hf:
    hf.create_dataset("RAEF",  data=dist_arr)

In [12]:
data=np.transpose(dist_arr, (1,2, 0))
data.shape, data.ndim

((15, 15, 246225), 3)

In [13]:
with h5py.File("RAEF_50_Run1_data.h5", "r") as f:
    contact_maps = f["RAEF"][...]

print ("contact_map_shape",contact_maps.shape) 

coo_matrices = [coo_matrix(cm) for cm in contact_maps]
X = [np.concatenate((c.row, c.col)) for c in coo_matrices]
X = np.array(X, dtype=object)
print(X.astype(np.float64).dtype)

X.shape, print(X.dtype) 


contact_map_shape (246225, 15, 15)


In [ ]:
import h5py
h= h5py.File('RAEF_data_coo_mat.h5', 'w')
dset=h.create_dataset("RAEF",  data=X)
h5py.close()

In [ ]:
x = np.arange(200).reshape((4,5,10))
x.shape
with open(Name+"_contact_data.txt", 'w') as outfile: 
    outfile.write('# Array shape: {0}\n'.format(data.shape))
    for data_slice in data:
        np.savetxt(outfile, data_slice, fmt='%-7.2f')


In [ ]:
data=np.loadtxt('RAEF_50_Run1_contact_data.txt')
new_data = data.reshape((15,15,3675))
data.shape, new_data.shape